# e-magyar elemzés

---

(2021. 04. 16.)

Mittelholcz Iván

## 1. Az e-magyar használata

Az elemzendő szöveg:

In [ ]:
!cat orkeny.txt

Az e-magyar legfrissebb verziójának letöltése:

In [ ]:
!docker pull mtaril/emtsv:latest

Az *orkeny.txt* elemzése, az eredmény kiírása az *orkény.tsv* fájlba:

In [ ]:
!docker run --rm -i mtaril/emtsv:latest tok,morph,pos,ner,conv-morph,dep <orkeny.txt >orkeny.tsv

Magyarázatok:

- `!docker run --rm -i mtaril/emtsv:latest`: Az *e-magyar* futtatása
- `tok,morph,pos,ner`: a használt modulok felsorolása
    - `tok`: tokenizálás
    - `morph`: morfológiai elemzés
    - `pos`: szófaji egyértelműsítés
    - `ner`: névelem felismerés
- `<orkeny.txt`: Az elemzendő szöveg beolvasása az *orkeny.txt* fájlból.
- `>orkeny.tsv`: Az elemzés kiírása az *orkeny.tsv* fájlba.

## 2. Az elemzés beolvasása *pandas DataFrame*-be

A TSV fájl beolvasásánál használt új paraméterek:

- `dtype=str`: A stringet tartalmazó cellákat alapból is stringnek szokta értelmezni a pandas, de ha biztosra akarunk menni, nem árt, ha kifejezetten megkérjük erre.
- `keep_default_na=False`: Ha ezt *False*-ra állítjuk, meghagyja az üres stringeket üres stringeknek és nem fogja azokat *NaN*-ként értelmezni. Ez a *wsafter* sor helyes beolvasásához kell.
- `skip_blank_lines=False`: A *Pandas* alapból átugorja az üres sorokat. Az e-magyar viszont az üres sorokat használja a mondatok elhatárolására, ezért meg kell mondani a *Pandas*-nak, hogy ne dobja ki az üres sorokat.

Részleteket a `df.read_csv()` [dokumentációjában](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

In [ ]:
import pandas as pd

df = pd.read_csv('orkeny.tsv', sep='\t', dtype=str, keep_default_na=False, skip_blank_lines=False)
df.head(50)

Oszlopok:

- *form*: A tokenizáló modul (*tok*) kimenete. A szövegben található tokeneket (szóalakokat, írásjeleket) tartalmazza.
- *wsafter*: Szintén a tokenizáló kimenete. A tokenek után található *whitespace* karaktereket tartalmazza.
- *anas*: A morfológiai elemző kimenete (*morph*). Szögletes zárójelpáron belül tartalmazza a lehetséges morfológiai elemzések listáját. Használt morfológiai kódok leírása [itt](https://e-magyar.hu/hu/textmodules/emmorph_codelist).
- *lemma*: A szófaji egyértelműsítő kimenete (*pos*). A legvalószínűbb morfológiai elemzéshez tartozó lemmát tartalmazza. 
- *xpostag*: Szintén a szófaji egyértelműsítő kimenete (*pos*). A legvalószínűbb morfológiai elemzést tartalmazza. 
- *NER-BIO*: A tulajdonnév felismerő modul kimenete (*ner*). Leírása [itt](https://e-magyar.hu/hu/textmodules/emner).

## 3. Elemzesek


### 3.1. Felhasználási esetek

#### Feladatok, amikhez egyszerre elég egy sort (*row*) figyelembe venni

- Szűrni bizonyos pos-tagekre, pl. keressük a múltidejű igéket.
- Adott lemmahalmaz múltidejű előfordulásai.
- Több morfológiai jegy figyelembevétele: pl. adott lemmahalmaz múltidejű előfordulásai egyeszám elsőszemélyben ill. többesszám elsőszemélyben.

A végén számolni kéne ezeket: az összes tokenszámhoz, vagy szószámhoz, vagy az összes igéhez képest milyen arányban fordulnak elő ezek az alakok.

#### Feladatok, amikhez több sort kell figyelembe venni

- Van-e személyes névmás az ige mellett? Pl. "éldegéltem" vs. "én éldegéltem".
- Főnévnek van-e jelzője?
- Igének van-e határozószava?
- Tagmondat szintű elemzés: keressük azon tagmondatokat, amikben van kötőszó, de nincs múltidejű igealak.

Ezeket megint arányítani kell: az összes főnévből mennyinek van jelzője, az összes igéből mennyinek van határozója.

#### Feladatok, amikhez az eredeti szöveget kell módosítani

- Potenciálisan többszavas kifejezések keresése ([emterm](https://github.com/dlt-rilmta/emterm)!).
- Szövegbe visszaírni elemzések eredményét XML-szerűen, pl. <érzelmi_kifejezés>...</érzelmi_kifejezés>

### 3.2. Egy soros feladatok megoldása

In [ ]:
# multideju igek aranya

def is_not_punct(row):
    pos = row['xpostag']
    return not pos.startswith('[Punct]')

def is_verb(row):
    pos = row['xpostag']
    return pos.startswith('[/V]')

def is_past_verb(row):
    pos = row['xpostag']
    return pos.startswith('[/V][Pst.')

mask0 = df.apply(is_not_punct, axis=1)
mask1 = df.apply(is_verb, axis=1)
mask2 = df.apply(is_past_verb, axis=1)

count_word = len(df[mask0])
count_verb = len(df[mask1])
count_past_verb = len(df[mask2])

print('multideju igek / osszes token: ', count_past_verb/len(df))
print('multideju igek / osszes szo: ', count_past_verb/count_word)
print('multideju igek / osszes ige: ', count_past_verb/count_verb)
df[mask2]

In [ ]:
# egyesszam 3. szemelyu igek

def is_3sg_verb(row):
    pos = row['xpostag']
    return pos.startswith('[/V]') and '3Sg' in pos

mask3 = df.apply(is_3sg_verb, axis=1)

count_3sg_verb = len(df[mask3])

print('3sg igek / osszes token: ', count_3sg_verb/len(df))
print('3sg igek / osszes szo: ', count_3sg_verb/count_word)
print('3sg igek / osszes ige: ', count_3sg_verb/count_verb)
df[mask3]

In [ ]:
# adott lemmahalmaz keresése

def is_lemma_in_set(row):
    lemma = row['lemma']
    lemmaset = {'iszik', 'van'}
    pos = row['xpostag']
    is_in_lemmaset = lemma in lemmaset
    is_3sg = '3Sg' in pos
    return is_in_lemmaset and is_3sg

mask4 = df.apply(is_lemma_in_set, axis=1)

count_lemmaset = len(df[mask4])

print('halmazban levo igek / osszes token: ', count_lemmaset/len(df))
print('halmazban levo igek / osszes szo: ', count_lemmaset/count_word)
print('halmazban levo igek / osszes ige: ', count_lemmaset/count_verb)

df[mask4]

### 3.3. Több soros feladatok megoldása

Algoritmus: Ha csak egy elem távolságba kell ellátni, akkor érdemes egy segédváltozóban eltárolni a ciklus előző elemének az értékét, vagy a vele kapcsolatos feltétel értékét.

In [ ]:
# Rávezetés 1.: Keressük a maganhángzóval kezdődő gyümölcsöket.
# elvárt eredmény: ['alma', 'eper']
l = ['alma', 'barack', 'citrom', 'dinnye', 'eper', 'füge']

result = []
for word in l:
    if word[0] in {'a', 'e', 'i', 'o', 'u'}:
        result.append(word)
print(result)

In [ ]:
# Rávezetés 2.: Menjünk végig egy listán úgy, hogy az aktuális elem mellett írjuk ki az előzőt is.
# Az első sorban az előző elem hiányozni fog.
l = ['alma', 'barack', 'citrom', 'dinnye', 'eper', 'füge']

previous = ''
for current in l:
    print(previous, current)
    previous = current # a ciklusmag végén mindig frissítjük az előző elemet az aktuálissal

In [ ]:
# Rávezetés 3.: Keressük azokat a gyümölcsöket, amik magánhangzóval kezdődő gyümölcs után következnek.
# elvárt eredmény: ['barack', 'fuge']
# A segédváltozóban nem az előző elemet tároljuk, csak azt, hogy az előző elem magánhangzóval kezdődőtt-e.

l = ['alma', 'barack', 'citrom', 'dinnye', 'eper', 'füge']

result = []
previous_startswith_vowel = False
for current in l:
    if previous_startswith_vowel:
        result.append(current)
    previous_startswith_vowel = current[0] in {'a', 'e', 'i', 'o', 'u'}
print(result)

Hogy a fentieket alkalmazni tudjuk *DataFrame* esetében is, ahhoz végig kell tudnunk iterálni a *DataFrame* sorain. Ezt az [`.iterrows()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) metódust használva tudjuk megtenni. A metódus minden sort egy *tuple*-ként ad vissza, aminek az első eleme az *index* (sorszám), a második a sor maga, mint [*Series*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html#pandas.Series).

In [ ]:
# Van-e névelő a főnév előtt?

def is_noun(row):
    return row['xpostag'].startswith('[/N')

mask5 = df.apply(is_noun, axis=1)

mask6 = []
is_prev_article = False
for index, row in df.iterrows():
    is_current_noun = row['xpostag'].startswith('[/N')
    mask6.append(is_current_noun and is_prev_article)
    is_prev_article = row['xpostag'] in {'[/Det|Art.Def]', '[/Det|Art.NDef]'}

print('névelős főnév / összes főnév: ', len(df[mask6])/len(df[mask5]))
    
#df['noun_with_article'] = mask5
#df.head(50)
df[mask6]

Algoritmus: Ha nem csak a szomszédos elemet kell látnunk, hanem elemet is, akkor érdemes egy *ablakkal* (*frame*-mel) végigmenni a listán.

In [ ]:
# Rávezetés 1.: Menjünk végig egy 3 elemet tartalmazó ablakkal a listán.

l = ['alma', 'barack', 'citrom', 'dinnye', 'eper', 'füge']

length = 3
frame = []

for i in l:
    frame.append(i)
    if len(frame) < length: # meg tul rovid a frame
        continue
    if len(frame) > length: # mar tul hosszu a frame
        frame.pop(0)
    print(frame)

In [ ]:
# Rávezetés 2.: Ál-elemekkel kiegészített lista.
# Ha a frame-ek első elemei vizsgáljuk (mert arra vagyunk kíváncsiak, van-e utána olyan, ami érdekes),
# akkor a fenti módon sosem jutunk oda, hogy az 'eper' vagy a 'füge' első elem legyen.
# Ha a frame-ek utolsó elemeit vizsgáljuk (mert arra vagyunk kíváncsiak, van-e előtte olyan, ami érdekes),
# akkor a fenti módon sosem jutunk oda, hogy az 'alma' vagy a 'barack' utolsó elem legyen.
# Az első esetben a lista végét kell kiegészíteni álelemekkel (None),
# a második esetben a lista elejére kell álelemeket beszúrni.


l = ['alma', 'barack', 'citrom', 'dinnye', 'eper', 'füge']

# álelemek a lista végén
length = 3
frame = []

for i in l + [None]*(length-1):
    frame.append(i)
    if len(frame) < length: # meg tul rovid a frame
        continue
    if len(frame) > length: # mar tul hosszu a frame
        frame.pop(0)
    print(frame)

print('--------')

# álelemek a lista elején
length = 3
frame = []

for i in [None]*(length-1) + l:
    frame.append(i)
    if len(frame) < length: # meg tul rovid a frame
        continue
    if len(frame) > length: # mar tul hosszu a frame
        frame.pop(0)
    print(frame)

In [ ]:
# Rávezetés 3.: Keressük azokat az elemeket, amik után az első vagy második elem magánhangzóval kezdődik.
# Az aktuális elemtől jobbra keresünk bizonyos tulajdonságú elemeket --> a listát jobbról egészítjük ki álelemekkel.

l = ['alma', 'barack', 'citrom', 'dinnye', 'eper', 'füge']

length = 3
frame = []
vowels = {'a', 'e', 'i', 'o', 'u'}

result = []
for i in l + [None] * (length -1):
    frame.append(i)
    if len(frame) < length:
        continue
    if len(frame) > length:
        frame.pop(0)
    for x in frame[1:]:
        if x is None: # ha None-ba botlunk, akkor skippeljük
            continue
        if x[0] in vowels:
            result.append(frame[0])
        
print(result)

In [ ]:
# Feladat: keressük az igekötők után lévő igéket.
# Az eredményből csak az ('El', 'patkoltak') pár lesz a jó. Finomítás később.

length = 10
frame = []
result = []
mylist = [row for index, row in df.iterrows()] + [None] * (length - 1)

# vegigmegyunk az álelemekkel kiegészített sorokon
for row in mylist:
    # frissitjuk a frame-et
    frame.append(row)
    if len(frame) < length:
        continue
    if len(frame) > length:
        frame.pop(0)
    # igekoto-e az elso elem? Ha igen, akkor megnezzuk, hogy utana valamelyik szo ige-e
    if frame[0]['xpostag'] == '[/Prev]':
        for frow in frame[1:]: # A frame-beli sorokat frow-nak nevezzuk el.
            if frow is None:
                continue
            if frow['xpostag'].startswith('[/V]'):
                # iget talaltunk, igekotot + iget hozzaadjuk az eredmenyhez
                result.append((frame[0]['form'], frow['form']))
                break # megvan az ige, abbahagyjuk a keresest
print(result)

In [ ]:
# Finomítás: Mondathatár után ne keressünk igét, mert az biztos nem az előző mondat igekötőjéhez fog tartozni.
# A mondathatárt a TSV üres sora jelöli. Ez a dataframe-ben olyan sor lesz, amiben minden cella egy üres string.
# (Elég a "form" cellát ellenőrizni, az nem lehet üres.)
# Az eredményekből a ('meg', 'akadt') pár még mindig rossz. Ezt a frame rövidebbre vételével lehet kiszűrni.

length = 10
frame = []
result = []
mylist = [row for index, row in df.iterrows()] + [None] * (length - 1)

# vegigmegyunk az álelemekkel kiegészített sorokon
for row in mylist:
    # frissitjuk a frame-et
    frame.append(row)
    if len(frame) < length:
        continue
    if len(frame) > length:
        frame.pop(0)
    # igekoto-e az elso elem? Ha igen, akkor megnezzuk, hogy utana valamelyik szo ige-e
    if frame[0]['xpostag'] == '[/Prev]':
        for frow in frame[1:]:
            if frow is None:
                continue
            # Mondathatár vizsgálata: ha a form nem tartalmaz semmit, akkor utána új mondat jön.
            if len(frow['form']) == 0:
                break
            if frow['xpostag'].startswith('[/V]'):
                # iget talaltunk, igekotot + iget hozzaadjuk az eredmenyhez
                result.append((frame[0]['form'], frow['form']))
                break # megvan az ige, abbahagyjuk a keresest
print(result)

In [ ]:
# Finomítás: A feladat ugyan az, mint az elobb, de most uj oszlopot csinalunk a dataframe-nek.
# Az uj oszlop default egy kotojelet tartalmaz, de az igekotoknel a feltetelezett iget irjuk bele.

length = 3
frame = []
result = []
mylist = [row for i, row in df.iterrows()] + [None] * (length - 1)

for row in mylist:
    frame.append(row)
    if len(frame) < length:
        continue
    if len(frame) > length:
        frame.pop(0)
    res = '-'
    if frame[0]['xpostag'] == '[/Prev]':
        for frow in frame[1:]:
            if frow is None:
                continue
            if len(frow['form']) == 0:
                continue
            if frow['xpostag'].startswith('[/V]'):
                res = frow['lemma']
                break
    result.append(res)

df['preverb'] = result
# kiirjuk a kerdeses reszt
df.iloc[120:128, :]

### 3.4. Elemzés visszaírása az eredeti szövegbe

In [ ]:
# eredeti szöveg kiírása:
# - minden sor 'form' és 'wsafter' celláját összeragasztjuk és hozzáadjuk ez eredmény listához
# - az eredmény lista elemeit a join metódussal egyesítjük egyetlen szöveggé
# - a szövegben lévő '\\n'-eket lecseréljük igazi sortörésekre 

text = []
for index, row in df.iterrows():
    text.append(row['form'] + row['wsafter'])
text = ''.join(text)
text = text.replace('\\n', '\n')
print(text)

In [ ]:
# Feladat: NER-BIO oszlop xml-esítése.
# Itt is a form és wsafter cellákat ragasztjuk össze és adjuk egy listához, de nézzük a ner cellákat is.
# - ha egy ner cella B-vel kezdődik (pl. B-ORG), akkor nyitunk egy <ORG> címkét és csak utána írjuk a form cellát.
# - ha egy ner cella E-vel kezdődik (pl. E-ORG), akkor a form cella után lezárjuk a címkét (</ORG>)
# - a szövegben nincs példa az egy elemű címkékre (pl. 1-ORG), de azt is kezeljük

text = []
for index, row in df.iterrows():
    form = row['form']
    ws = row['wsafter']
    ner = row['NER-BIO']
    if ner.startswith('B'):
        # named entity kezdodik, xml tag-et nyitunk:
        form = f'<{ner[2:]}>{form}'
    elif ner.startswith('E'):
        # named entity vegzodik, xml tag-et zarunk:
        form = f'{form}</{ner[2:]}>'
    elif ner.startswith('1'):
        # egy elemu named entity, xml tag-ebe tesszuk:
        form = f'<{ner[2:]}>{form}</{ner[2:]}>'
    text.append(form+ws)
text = ''.join(text)
text = text.replace('\\n', '\n')
print(text)
